In [2]:
#Course_code
import xml.etree.cElementTree as ET
from collections import defaultdict
import re

In [ ]:
osm_file = open("chicago_abbrev.osm","r")

In [ ]:
street_type_re = re.compile(r'\S+\.?$',re.IGNORECASE) #extract street info.
street_types = defaultdict(int)

In [ ]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type] +=1

In [ ]:
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys,key=lambda s:s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" %(k, v)

In [ ]:
def is_street_name(elem):
    return (elem.tag =="tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
    print_sorted_dict(street_types)
    
if __name__ =='__main__':
    audit()

In [ ]:
#auditing a cross_field constraint
def ensure_float(v):
    if is_numnber(v):
        return float(v)
    
def audit_population_density(input_file):
    for row in input_file:
        population = ensure_float(row['populationTotal'])
        area = ensure_float(row['areaLand'])
        population_density = ensure_float(row['populationDensity'])
        if population and area and population_density:
            calculated_density = population / area
            if math.fabs(calculated_density - population_density) >10:
                print "Possibly bad population density for ", row['name']
if __name__=='__main__':
    input_file = csv.DictReader(open("cities.csv"))
    skip_lines(input_file, 3)
    audit_population_density(input_file)

In [ ]:
#Quiz
import csv
import pprint

Input_File = 'auto.csv'
Output_Good = 'autos-valid.csv'
Output_Bad = 'fixme-autos.csv'

def process_file(input_file, output_good, output_bad):
    #store data into output lists
    data_good = []
    data_bad = []
    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        for row in reader:
            #validate URI value
            if row['URI'].find("dbpedia.org") <0:
                continue
            
            ps_year = row['productionStartYear'][:4]
            try:# use try/except to filter valid items
                ps_year = int(ps_year)
                row['productionStartYear'] = ps_year
                if (ps_year >=1886) and (ps_year <=2014):
                    data_good.append(row)
                else:
                    data_bad.append(row)
            except ValueError: #non-numeric strings caught by exception
                if ps_year =='NULL':
                    data_bad.append(row)
                
        
#write outputs
    with open(output_good, "w") as g:
        writer = csv.DictWriter(g, delimiter= ",", fieldnames = header)
        writer.writeheader()
        for row in data_good:
            writer.writerow(row)
            
    with open(output_bad, "w") as b:
        writer = csv.DictWriter(b, delimiter= ",", fieldnames = header)
        writer.writeheader()
        for row in data_bad:
            writer.writerow(row)
            
def test():
    process_file(Input_File, Output_Good, Output_Bad)
    
if __name__=="__main__":
    test()


In [ ]:
#course code
#Audting accuracy
client = MongoClient('mongodb://localhost:27017')
db = client.examples

def skip_lines(input_file, skip):
    for i in range(0,skip):
        next(input_file)
def audit_country(input_file):
    for row in input_file:
        country = row['country_label']
        country = country.strip()
        if (country == "NULL") or (country = ""):
            continue
        if db.countries.find({"name":country}).count()!=1:
            print "Not found:", country
            
if __name__=="__main__":
    input_file = csv.DictReader(open("cities.csv"))


In [ ]:
#auditing uniformity
import csv
import pprint

filename = "wgs84_pos#lat"
minval = -90
maxval = 90

def skip_lines(input_file, skip):
    for i in range(0,skip):
        next(input_file)

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def audit_float_field(v, counts):
    v = v.strip()
    if v =="NULL":v
        counts['nulls']+=1
    elif v =="":
        counts['empties']+=1
    elif is_array(v):
        counts['arrays']+=1
    elif not is_number(v):
        print "Found non number:", v
    else:
        v = float(v)
        if not ((minval < v) and (v <maxval)):
            print "Found out of range value", v
        
    

if __name__=="__main__":
    input_file = csv.DictReader(open("cities3.csv"))
    skip_lines(input_file,3)
    counts = {"nulls":0, "empties":0, "arrays":0}
    nrow = 0
    for row in input_file:
        audit_float_field(row[fieldname],counts)
        nrow +=1
    print "num cities:",nrow
    print "nulls:",counts['null']
    print "empties:",counts['empties']
    print "arrays:",counts['arrays']
    